# Проект "Выбор локации для скважины"

# Введение

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три значимых признака точек;
- product — объём запасов в скважине (тыс. баррелей).

# Условия задачи

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
display(df0.head())
display(df0.info())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [4]:
df1 = pd.read_csv('/datasets/geo_data_1.csv')
display(df1.head())
display(df1.info())

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [5]:
df2 = pd.read_csv('/datasets/geo_data_2.csv')
display(df2.head())
display(df2.info())

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Удалим столбец **id**, поскольку линейная регрессия работает только c числовыми данными int и float. На модели это не скажется, поскольку данная графа не содержит ни количественные, ни категориальные переменные

In [6]:
df0 = df0.drop(['id'], axis=1)
df1 = df1.drop(['id'], axis=1)
df2 = df2.drop(['id'], axis=1)

## Обучение и проверка модели

In [7]:
features_0 = df0.drop(['product'], axis=1)
target_0 = df0['product']

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_valid_0 = model_0.predict(features_valid_0)

result_0 = mean_squared_error(target_valid_0, predictions_valid_0)**0.5 
print("RMSE модели линейной регрессии на валидационной выборке 0:", result_0)

RMSE модели линейной регрессии на валидационной выборке 0: 37.5794217150813


In [8]:
features_1 = df1.drop(['product'], axis=1)
target_1 = df1['product']

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)

result_1 = mean_squared_error(target_valid_1, predictions_valid_1)**0.5 
print("RMSE модели линейной регрессии на валидационной выборке 1:", result_1)

RMSE модели линейной регрессии на валидационной выборке 1: 0.893099286775616


In [9]:
features_2 = df2.drop(['product'], axis=1)
target_2 = df2['product']

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)

result_2 = mean_squared_error(target_valid_2, predictions_valid_2)**0.5 
print("RMSE модели линейной регрессии на валидационной выборке 2:", result_2)

RMSE модели линейной регрессии на валидационной выборке 2: 40.02970873393434


Лучший RMSE на валидационной выборке региона 1

## Подготовка к расчёту прибыли

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [10]:
BUDGET = 10000000000
INCOME_PER_UNIT = 450000
COUNT=200

budget_per_well = BUDGET / COUNT
min_vol = budget_per_well / INCOME_PER_UNIT
min_vol

111.11111111111111

In [11]:
print('Средний обьем в 0 регионе:', predictions_valid_0.mean())
print('Средний обьем в 1 регионе:', predictions_valid_1.mean())
print('Средний обьем в 2 регионе:', predictions_valid_2.mean())

Средний обьем в 0 регионе: 92.59256778438038
Средний обьем в 1 регионе: 68.728546895446
Средний обьем в 2 регионе: 94.96504596800489


Средний объём сырья, достаточный для безубыточной разработки, превышает предсказанные средние запасы в каждом из регионов, в этой связи нет возможности выделить лучший регион для возможной разработки новых скважин. 

## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- Выберем скважины с максимальными значениями предсказаний.
- Сумммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
- Рассчитаем прибыль для полученного объёма сырья.

In [15]:
def profit(target, predict):
    predict_sorted = pd.Series(predict).sort_values(ascending=False)
    selected = target[predict_sorted.index][:COUNT]
    prof = (selected.sum() * INCOME_PER_UNIT) - BUDGET
    return(prof)
print('Прибыль в регионе 0:', profit(target_valid_0, predictions_valid_0))
print('Прибыль в регионе 1:', profit(target_valid_1, predictions_valid_1))
print('Прибыль в регионе 2:', profit(target_valid_2, predictions_valid_2))

Прибыль в регионе 0: -8238159977.365687
Прибыль в регионе 1: -8698530691.175531
Прибыль в регионе 2: -7870007760.737813


Посчитаем риски и прибыль для каждого региона:
- Применим технику **Bootstrap** с 1000 выборок, чтобы найти распределение прибыли.
- Найем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Риск убытков - это доля отрицательных ответов, укажем их в процентах.

In [20]:
def count_profit (target, predict):
    state = np.random.RandomState(100)
    values = []
    for i in range(1000):
        target_sample = pd.Series(target).reset_index(drop=True).sample(n=500, random_state=state,replace=True)
        prob_sample = pd.Series(predict)[target_sample.index]
        rev_sample = profit(target_sample, prob_sample)
        values.append(rev_sample)
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    count = 0
    for i in range(len(values)):
        if values[i] < 0:
            count += 1
    return values.mean(), lower, upper, (values < 0).mean()*100 #count/len(values)*100

print('Cредняя прибыль, интервал и риск убытков в регионе 0:', count_profit(target_valid_0, predictions_valid_0))
print('Cредняя прибыль, интервал и риск убытков в регионе 1:', count_profit(target_valid_1, predictions_valid_1))
print('Cредняя прибыль, интервал и риск убытков в регионе 2:', count_profit(target_valid_2, predictions_valid_2))

Cредняя прибыль, интервал и риск убытков в регионе 0: (408417421.9472264, -122256880.33374405, 989778917.6915294, 6.800000000000001)
Cредняя прибыль, интервал и риск убытков в регионе 1: (504932744.04616815, 95193084.76780239, 916496786.6306496, 1.0)
Cредняя прибыль, интервал и риск убытков в регионе 2: (418317134.1329915, -159796161.96567866, 947337201.5616112, 7.8)


Вывод: Лучший регион для разработки скважин = Регион 1 (/datasets/geo_data_1.csv)